In [1]:

import cv2 as cv
import math
import numpy as np
from matplotlib import pyplot as plt
import random as rng
import glob
from IPython.display import Image, display
import random 

sift = cv.SIFT_create()
bf1 = cv.BFMatcher()
bf = cv.BFMatcher(cv.NORM_HAMMING, crossCheck=True)
orb = cv.ORB_create()
pad=1000
showall=False
showsome=True
scale=0.2
l3=True
l2=True
l4=True
folder='james'
flann = cv.FlannBasedMatcher()

In [2]:
imgData = {
  "kp": [],
  "d": [],
    'skp':[],
    'sd':[],
  "image": []
}

matchData = {
  "matches": [],
    'matrix':[],
  "score": [],
}


pairData = {
  "idx": 0,
  "score": 1000,
  "matchdata":matchData,
    "matrix1":[],
    "matrix2":[]
}

info = {
  "matches":[],
    "score":[],
    "matrix":[],
      "imgdata": [],
  "pairs": [],
    'total':0
}
inimg = []
inimg2 = []

In [3]:
def reset():
    inimg = []
    inimg2 = []
    imgData = {
  "kp": [],
  "d": [],
    'skp':[],
    'sd':[],
  "image": []
    }

    matchData = {
  "matches": [],
    'matrix':[],
  "score": [],
    }


    pairData = {
  "idx": 0,
  "score": 1000,
  "matchdata":matchData,
    "matrix1":[],
    "matrix2":[]
    }

    info = {
  "matches":[],
    "score":[],
    "matrix":[],
      "imgdata": [],
  "pairs": [],
    'total':0
    }

In [4]:
def getKeyp(folder):
    d=[]
    kp=[]
    img=[]
    xa=[]
    xa1=[]
    a=0

    print('finding data...')
    il=[]
    for imageName in glob.glob(folder+'/*.jpg'):
        a=a+1
        i1 = cv.imread(imageName)
        i1=cv.resize(i1, (int(i1.shape[1]*scale),int(i1.shape[0]*scale)), interpolation = cv.INTER_AREA)
        i1=addPad(i1).copy()
#         kp1, d1 = orb.detectAndCompute(i1,None)#key points and detectors
        kp2, d2 = sift.detectAndCompute(i1,None)#key points and detectors
        imgData['kp']=kp2
        imgData['d']=d2
#         imgData['skp']=kp1
#         imgData['sd']=d1
#         imgData['skp']=kp2
#         imgData['sd']=d2
        imgData['image']=i1.copy()
    for i in range(a):
        xa1.append(0)
        xa.append([])
        info['pairs'].append([])
    for i in range(a):
        info['matrix'].append(xa.copy())
        info['score'].append(xa1.copy())
        info['matches'].append(xa.copy())
    info['total']=a
    print('data found')
#     print(imgData)
#     print(info)
    info['imgdata']=img
    return info.copy()

In [5]:
def getData(d1,d2,k1,k2,i,j,d11,d22):
    s=0
    qu=[]
    tr=[]
    kp1=[]
    kp2=[]
    thresh=30
    good_points =[]
    matches=[]
    bmatches=[]
    matches = flann.knnMatch(d1,d2,k=2)
#     bmatches = bf.match(d11,d22)
#     print(matches)
#     orderedmatches = sorted(bmatches, key = lambda x:x.distance)
    th=0.2
    if folder == 'room' or folder == 'map':
        th=0.5
    for m,n in matches:
#         if 2*orderedmatches[0].distance>thresh/2:
        s=s+m.distance      
        if(m.distance < th*n.distance):
            good_points.append(m)

    info["matches"][i][j]=good_points.copy()
#     info["matches"][j][i]=good_points.copy()
    
    info["score"][i][j]=(s/len(matches))
    good_points=[]
    return (s/len(matches))

In [6]:
def getPairs(folder):
    info=getKeyp(folder)
#     print(info)
    data=[]
    pairs=[]
    pair=[]
    pair1=[]
    matchData=[]
    thresh=255
    if folder == 'office':
        thresh=290
    elif folder == 'test':
        thresh = 250
    elif folder == 'james':
        thresh=270
    elif folder == 'room':
        thresh=350
    elif folder == 'ac':
        thresh=270
    elif folder == 'map':
        thresh=295
    elif folder == 'turtle':
        thresh=295
    elif folder == 'WLH':
        thresh=300

    print('getting pairs...')            
    for i in range(0,info['total']-1):
        for j in range(0,info['total']):
            if not i == j:
                score= getData(info['imgdata'][i]["d"],info['imgdata'][j]["d"],info['imgdata'][i]["kp"],info['imgdata'][j]["kp"],i,j,info['imgdata'][i]["sd"],info['imgdata'][j]["sd"])
#             print(info["score"])
                info["score"][i][j]=score
#             print(info["score"])
            #             print(matchData["score"])
#             office:20
#             james:25    

#             print(score)
                if(score<thresh)and len(info['matches'][i][j])>6:
#                 pair.append(j)
#                 pair1.append(i)
                    info['pairs'][i].append(j)
#                 info['pairs'][j].append(i)
        pair=[]
        pair1=[]
    print('pairs found')
    return info.copy()

In [7]:
def  getMat():
    print("start mat finding...")
    mat = []
    tempmat = []
    M=[]
    for p in range(info["total"]-1):
        if len(info["pairs"][p]) ==0:
            print("***ERROR***")
            
        for i in range(len(info["pairs"][p])):
            xq=info["pairs"][p][i]
            p1 = np. float32([info['imgdata'][p]['kp'][m.queryIdx].pt for m in info['matches'][p][info['pairs'][p][i]]]).reshape(-1,1,2)
            p2 = np. float32([info['imgdata'][info["pairs"][p][i]]['kp'][m.trainIdx].pt for m in info['matches'][p][info['pairs'][p][i]]]).reshape(-1,1,2)     
            info['matrix'][p][info["pairs"][p][i]] = cv.findHomography(p2, p1, cv.RANSAC,5.0)[0]
        
    return

In [8]:
def saveMe(p,i):
    print(p)
    print(i)
    p1 = np. float32([info['imgdata'][p]['kp'][m.queryIdx].pt for m in info['matches'][p][i]]).reshape(-1,1,2)
    p2 = np. float32([info['imgdata'][i]['kp'][m.trainIdx].pt for m in info['matches'][p][i]]).reshape(-1,1,2)
    M, mask = cv.findHomography(p2, p1, cv.RANSAC,5.0)
    print("Saved fewf...")
    return M

In [9]:
def getCenter():
    min = 0
    mini=0
    sum=0
    sums=[]
    avglen=0
    for i in range(info['total']-1):
        sum=0
        for p in info['score'][i]:
#             print(p)
            sum=sum+p
        tem = len(info['score'][i])
        avglen=avglen+len(info['score'][i])
        if tem ==0:
            tem=1
            sum=1000000
            
        sums.append(sum/tem)    
            
#         if len(info['pairs'][i])==0:
#             sums.append(1000)
#         else:
#             sums.append(sum/len(info['pairs'][i]))\
    avglen=avglen/info['total']
    for i in range(info['total']-1):
        if sums[i]<sums[mini]:
            mini=i
    return mini

In [10]:
def addimg(img1,img2):
    i=0
    j=0
    print("add start")
    print("")
#     print("img1 type")
#     print(type(img1))
#     print(type(img1[0][0]))
    x=img2.shape[0]-1
    y=img2.shape[1]-1
    if y > img1.shape[1]-1:
        y=img1.shape[1]-1
    if x > img1.shape[0]-1:
        x=img1.shape[0]-1
    xa=0
    js=0
#     temp=img2.copy().transpose()
#     while i <y:
#         if cv.countNonZero(temp[i]) > 0:
#             js=i
#             i=0
#             break
#         i=i+1
#     i=temp[i]
#     while i >js:
#         if cv.countNonZero(temp[i]) > 0:
#             y=i
#             i=0
#             break
#         i=i-1
        
    while i <x:
        if showall or showsome:
            if i % (int(x/4)-1)==0:
                print(str(xa)+'%...')
                xa=xa+25
        if cv.countNonZero(img2[i]) > 0:
            while j < y:
                if(img2[i][j].any()>0):
                    img1[i][j]=img2[i][j]           
                j=j+1
        i=i+1
        j=js
       
    print("add done")
    print("")
#     plt.imshow(img1),plt.show()
    return img1

In [11]:
def sim(c):
    temp=[]
    temp.append(c)
    for i in info['pairs'][c]:
        temp.append(i)
        for az in info['pairs'][i]:
            if not az in info['pairs'][c] and not az in inimg:
                temp.append(az)
    ret = []
    for i in range(info['total']):
        if not i in temp:
            ret.append(i)
    if len(ret)>0:
        print("")
        print("L3")
        print("")
    return ret

In [12]:
def blur(image):
    blurred_img = cv.GaussianBlur(image, (201, 201), 0)
    mask = np.zeros(image.shape, np.uint8)
    mask1 = np.zeros(image.shape, np.uint8)

    gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
#     print(cv.threshold(gray, 60, 255, cv.THRESH_BINARY))
    thresh = cv.threshold(gray, 2, 255, cv.THRESH_BINARY)[1]
    contours, hierarchy = cv.findContours(thresh.copy(), cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

    cv.drawContours(mask, contours, -1, (255,255,255),int(30*scale))
    output = np.where(mask==np.array([255, 255, 255]), mask1, image)
    return output.copy()

In [13]:
def newH(img1,img2):
    good_points=[]

    kp1, d1 = sift.detectAndCompute(img1,None)#key points and detectors
    kp2, d2 = sift.detectAndCompute(img2,None)#key points and detectors
    matches = flann.knnMatch(d1,d2,k=2)

    for m,n in matches:
        if(m.distance < 0.7*n.distance):
            good_points.append(m)
    p1 = np. float32([kp1[m.queryIdx].pt for m in good_points]).reshape(-1,1,2)
    p2 = np. float32([kp2[m.trainIdx].pt for m in good_points]).reshape(-1,1,2)
    H=cv.findHomography(p1, p2)[0]
    return H

In [14]:
def stitchimg(comp,i1,i2,flag,img):
    img1= comp.copy()
    img2=info['imgdata'][i2]['image'].copy()
    H=info['matrix'][i1][i2]
    if flag:
        img2=img.copy()
#         H=newH(img1,img2)
    
    if len(H) == 0:
        H=saveMe(i1,i2)
    print("stitching together:")
    print(i1)
    print(i2)
    print(" ")
    warped1 = cv.warpPerspective(blur(img2.copy()),H , (img2.shape[1],img2.shape[0]))
    if showall:
        plt.figure(figsize=(20,10))
        print('warped')
        plt.imshow(cv.cvtColor(warped1, cv.COLOR_BGR2RGB)),plt.show()    
    ret = addimg(img1.copy(),warped1).copy()
    
    return ret

In [15]:
def updatePad(img):
    s=pad
    r=0
    l=0
    top=0
    b=0
    t=img.copy().transpose() 
    for i in range(len(img.shape)-1):
        if cv.countNonZero(img[i]) > 0:
            b=i
            break
    x=0        
    for i in range(len(img.shape)-1,0,-1):
        
        if cv.countNonZero(img[i]) > 0:
            top=x
            break
        x=x+1
            
    for i in range(len(t.shape)-1):
        if cv.countNonZero(t[i]) > 0:
            r=i
            break
    x=0        
    for i in range(len(img.shape)-1,0,-1):
        if cv.countNonZero(t[i]) > 0:
            l=x
            break
#             print(l,r,top,b)
        x=x+1
    if l < s:
        l=s-l
    else:
        l=0

    if r < s:
        r=s-r
    else:
        r=0        
    if top < s:
        t=s-top
    else:
        top=0        
    if  b< s:
        b=s-b
    else:
        b=0        
#     print(b+top+img.shape[1])
    temp = np.zeros([b+top+img.shape[1],l+r+img.shape[0],3], np.uint8)
    translation_matrix = np.float32([[1,0,top],[0,1,l]])
    temp=cv.warpAffine(img.copy(),translation_matrix,(b+top+img.shape[1],l+r+img.shape[0]))
    return img.copy()

In [16]:
def addPad(img):
    ret=img.copy()
    row, col = ret.shape[:2]
    bottom = ret[row-2:row, 0:col]
    mean = cv.mean(bottom)[0]

#     bordersize = int((ret.shape[0]+ret.shape[1])/4)
#     print(int((img.shape[0]+img.shape[1])))
    bordersize=int(pad*1.5)
    b1=bordersize
    bt=b1
    bb=b1
    if folder=='office':
        b1=3*bordersize
        bt=int(bordersize/2)
        bb=int(bordersize/2)
    if folder == 'james':
        bt=int(bt/4 )
        bb=bb+int((3*bb)/4)
    border = cv.copyMakeBorder(ret,top=int(bt),bottom=int(bb),left=int(b1),right=int(b1 ),borderType=cv.BORDER_CONSTANT,value=[0, 0, 0])
    return border.copy()

In [17]:
def stitch():
    c=getCenter()
    remaining=sim(c)
    print(remaining)
#     remaining=[]
    print('center is:')
    print(c)
    retImg = info['imgdata'][c]['image'].copy()
    inimg.append(c)
    for i in info['pairs'][c]:
        inimg.append(i)
        temp1 = info['imgdata'][i]['image'].copy()
        
        
        for az in info['pairs'][i]:
#             if not l2:
#                 break
            temp12 = info['imgdata'][az]['image'].copy()
            if not az == c and  not az == i and l2 and not az in inimg:
                if (az in info['pairs'][c] and info['score'][i][az]>info['score'][c][az]) or not az in info['pairs'][c]:
                    if (az in info['pairs'][c] and info['score'][i][az]<info['score'][c][az]):
                        print('newStep**********')
#                         info['pairs'][c].remove(az)
                    inimg.append(az)
                   
                    for az1 in info['pairs'][az]:
#                     if not l3:
#                         break
                        if  not az1 in info['pairs'][c] and  not az1 == c and not az1 == i and l3:
                            inimg.append(az1)
                            temp11 = info['imgdata'][az1]['image'].copy()
                            for az2 in info['pairs'][az1]:
                                temp111 = info['imgdata'][az2]['image'].copy()
                                if  not az2 in info['pairs'][c] and  not az2 == c and not az2 == i and not az2 == az and not az2 in inimg and l4:
                                    temp11=stitchimg(temp11.copy(),az1,az2,True,temp111).copy()
                                    inimg.append(az2)
                                    if showall:
                                        print('temp x3')
                                        plt.figure(figsize=(20,10))
                                        plt.imshow(cv.cvtColor(temp11, cv.COLOR_BGR2RGB)),plt.show()
                            temp12=stitchimg(temp12.copy(),az,az1,True,temp11).copy()
                            
                            if showall:
                                print('temp x2')
                                plt.figure(figsize=(20,10))
                                plt.imshow(cv.cvtColor(temp12, cv.COLOR_BGR2RGB)),plt.show()
                                
                    temp1=stitchimg(temp1.copy(),i,az,True,temp12.copy()).copy()
                if showall or showsome:
                    print('temp')
                    plt.figure(figsize=(20,10))
                    plt.imshow(cv.cvtColor(temp1, cv.COLOR_BGR2RGB)),plt.show()
        
        retImg=stitchimg(retImg,c,i,True,temp1.copy()).copy()
        if showall or showsome:
            print('next step')
            plt.figure(figsize=(20,10))
            plt.imshow(cv.cvtColor(retImg, cv.COLOR_BGR2RGB)),plt.show()
    plt.figure(figsize=(20,10))
    plt.imshow(cv.cvtColor(retImg, cv.COLOR_BGR2RGB)),plt.show()
    print("DONE!")
    return retImg
            

In [18]:
l2=True
l3=True
l4=True
showall=True
folder='WLH'
pad = 2000
scale=0.2
reset()
#************************************************************************
info = getPairs(folder)#**************************************************
#************************************************************************

finding data...
data found
getting pairs...


IndexError: list index out of range

In [ ]:
print("start matrix")
getMat()
print(info['pairs'])
print("done matrix")
# print(info)

In [ ]:
print(info['score'])

In [ ]:
showall=False
x=stitch()
print(inimg)

In [ ]:
inimg = []
inimg2 = []
imgData = {
  "kp": [],
  "d": [],
    'skp':[],
    'sd':[],
  "image": []
    }

matchData = {
  "matches": [],
    'matrix':[],
  "score": [],
    }


pairData = {
  "idx": 0,
  "score": 1000,
  "matchdata":matchData,
    "matrix1":[],
    "matrix2":[]
    }

info = {
  "matches":[],
    "score":[],
    "matrix":[],
      "imgdata": [],
  "pairs": [],
    'total':0
    }
pad=3000
scale=0.3
l3=True
l4=False
showall=False
folder='james'
info = getPairs(folder)
getMat()
print(info['pairs'])
x=stitch()
print(inimg)